In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./2.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../00.Tools')
                from crawler_configuration import get_header, get_proxy

                resp = requests.get(input_.loc[a, 'Handle'],
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                import json

                json_ = json.loads(resp.strip())

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(json_['description'].strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_td = html.xpath('//table[@id="ctl00_dc_partInfo_grdPartDetails"]/tbody/descendant-or-self::td')

                # = = = = = = = = = = = = = = =

                dict_src = {}
                list_src = [f'''https:{image.strip()}''' for image in json_['images']]
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = src

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Part Number': '',
                                         'Type': '',
                                         'Status': json_['available'],
                                         'Title': json_['title'].strip(),
                                         'Supplier': json_['vendor'].strip(),
                                         'Price': f'''$ {json_['price'] / 100:.2f}''',
                                         'Vehicle': '\n'.join([tag.strip() for tag in json_['tags']]),
                                         'Picture': '',
                                         'Url': f'''https://jaeautoelectrical.com.au{json_['url'].strip()}''',
                                         'Json_Src': json.dumps(dict_src)}])

                i = 0
                while i < len(list_td):
                    key, value = list_td[i].xpath('./text()')[0].strip(), list_td[i+1].xpath('./span/text()')[0].strip()
                    if key == 'Part No.':
                        df_temp.loc[0, 'Part Number'] = value
                    else:
                        df_temp.loc[0, key] = value
                    i += 2

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Handle': input_.loc[a, 'Handle']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Handle']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./3.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：385

[状态：ok，尝试次数：  1] - 10.https://jaeautoelectrical.com.au/products/a5623213-oex-air-conditioning-compressor-nissan.js
[剩余数量：373] - [当前时间：16:14:22]

[状态：ok，尝试次数：  1] - 2.https://jaeautoelectrical.com.au/products/a9331888-oex-air-conditioning-compressor.js
[剩余数量：372] - [当前时间：16:14:23]

[状态：ok，尝试次数：  1] - 5.https://jaeautoelectrical.com.au/products/a5623216-oex-air-conditioning-compressor-ford-mo.js
[剩余数量：371] - [当前时间：16:14:23]

[状态：ok，尝试次数：  1] - 3.https://jaeautoelectrical.com.au/products/a5623219-oex-air-conditioning-compressor-renault.js
[剩余数量：370] - [当前时间：16:14:23]

[状态：ok，尝试次数：  1] - 8.https://jaeautoelectrical.com.au/products/a5623220-oex-air-conditioning-compressor-nissan.js
[剩余数量：369] - [当前时间：16:14:24]

[状态：ok，尝试次数：  1] - 13.https://jaeautoelectrical.com.au/products/a5536856-oex-air-conditioning-compressor-12v-dire.js
[剩余数量：368] - [当前时间：16:14:24]

[状态：ok，尝试次数：  1] - 1.https://jaeautoelectrical.com.au/products/a9332209-oex-air-conditioning-compressor.js
[剩余数量：367] - [当前时间：16

KeyboardInterrupt
2024-07-10T08:14:27Z

KeyboardInterrupt



[状态：ok，尝试次数：  1] - 12.https://jaeautoelectrical.com.au/products/a5536857-oex-air-conditioning-compressor-24v-dire.js
[剩余数量：356] - [当前时间：16:14:27]

